In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,Flatten
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle

from keras import metrics
from keras.layers.merge import concatenate
import sys
sys.path.append('../../')
from Utilities.model_visualization import model_to_png

from PIL import Image # used for loading images
import numpy as np
from numpy import asarray
import matplotlib.pyplot as plt

import os # used for navigating to image path

from keras.layers import Input

import cv2
from keras.layers import concatenate
from keras.models import Model
from keras.applications import VGG16

import pandas as pd
from glob import glob
import tensorflow as tf

from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import StratifiedKFold
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.callbacks import EarlyStopping

import pickle
from scipy import misc
from keras import optimizers

from keras.utils import plot_model

from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import time

%matplotlib inline

Using TensorFlow backend.


<h1>Loading Data</h1>

In [2]:
df = pd.read_pickle('processed_df.pkl')
print(df.iloc[0])

text          Sunny days sweeping clouds away #Irma #PublicP...
text_info                                           informative
image_path    data_image/hurricane_irma/18_9_2017/9098755908...
damage                                              mild_damage
Name: 0, dtype: object


In [3]:
split_pos = int(df['text'].count()*0.8)
train = df[:split_pos]
test = df[split_pos:]

<h1>CNN Text Model</h1>

In [4]:
dataColumn = 'text'
labelColumn = 'text_info'

tags = train[labelColumn]
texts = train[dataColumn]

tags_Y = test[labelColumn]
texts_Y = test[dataColumn]

In [5]:
num_max = 1000
# preprocess
le = LabelEncoder()
tags = le.fit_transform(tags.astype(str))
tok = Tokenizer(num_words=num_max)
tok.fit_on_texts(texts)
mat_texts = tok.texts_to_matrix(texts,mode='count')

# For testing data
le_Y = LabelEncoder()
tags_Y = le_Y.fit_transform(tags_Y.astype(str))
tok_Y = Tokenizer(num_words=num_max)
tok_Y.fit_on_texts(texts_Y)
mat_texts_Y = tok.texts_to_matrix(texts_Y,mode='count')

In [6]:
# for cnn preproces
max_len = 100
cnn_texts_seq = tok.texts_to_sequences(texts)
cnn_texts_mat = sequence.pad_sequences(cnn_texts_seq,maxlen=max_len)

# For testing data
cnn_texts_seq_Y = tok.texts_to_sequences(texts_Y)
cnn_texts_mat_Y = sequence.pad_sequences(cnn_texts_seq_Y,maxlen=max_len)

In [7]:
filepath = "text_weights.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max', period=1)
early_stopping = EarlyStopping(monitor='val_acc', min_delta=0, patience=4, verbose=1, mode='max')
callbacks_list = [checkpoint, early_stopping]

In [8]:
def get_hybrid_model():    # Pre Trained Embeddings
    
    # load the whole embedding into memory
    embeddings_index = dict()
    f = open('Embeddings/glove.6B.100d.txt', encoding="utf8")
    for line in f:
        values = line.split()
        word = values[0]
        coefs = asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('Found %s word vectors.' % len(embeddings_index))
    
    # create a weight matrix for words in training docs
    embedding_matrix = np.zeros((len(tok.word_index) + 1, 100))
    for word, i in tok.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    #text classifier
    inputs = Input(shape=(100,))
    e = Embedding(len(tok.word_index) + 1,
                  100, 
                  weights=[embedding_matrix],
                  input_length=max_len, 
                  trainable=False)(inputs)
    x = Dropout(0.2)(e)
    x = Conv1D(128,
               3,
               padding='valid',
               activation='relu',
               strides=1)(x)
    x = GlobalMaxPooling1D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    hybrid_link = Dense(32, activation='relu', name='hybrid_link')(x)
    x = Dense(1, activation='sigmoid', name='Text_Classifier')(hybrid_link)
    text_classifier = x
    #image classifier
    IMAGE_SIZE = [224, 224]  # we will keep the image size as (64,64). You can increase the size for better results. 
    vgg = VGG16(input_shape = (224, 224, 3), weights = None, include_top = True)  # input_shape = (64,64,3) as required by VGG
    x = (vgg.layers[-2].output)
    image_model = Dense(4, activation = 'softmax')(x)  # adding the output layer with softmax function as this is a multi label classification problem.
    #hybrid model
    concatenate_layer = concatenate([image_model, hybrid_link]) 
    hybrid = Dense(4, activation='softmax', name='Hybrid_Classifier')(concatenate_layer)
    model = Model(inputs=[vgg.input, inputs], outputs=[hybrid,text_classifier])
    return model

In [9]:
def get_vgg_model():    # Pre Trained Embeddings
    
    #image classifier
    IMAGE_SIZE = [224, 224]  # we will keep the image size as (64,64). You can increase the size for better results. 
    vgg = VGG16(input_shape = (224, 224, 3), weights = None, include_top = True)  # input_shape = (64,64,3) as required by VGG
    x = (vgg.layers[-2].output)
    image_model = Dense(4, activation = 'softmax',name='Hybrid_Classifier')(x)  # adding the output layer with softmax function as this is a multi label classification problem.
    model = Model(inputs=[vgg.input], outputs=[image_model])
    return model

In [10]:
hybrid_model = get_hybrid_model()
vgg_model = get_vgg_model()
hybrid_model.compile(loss=['categorical_crossentropy','binary_crossentropy'],
                       optimizer= optimizers.adam(lr=0.00008),
                       metrics=['accuracy',metrics.mae, metrics.categorical_accuracy])
vgg_model.compile(loss='categorical_crossentropy',
                       optimizer= optimizers.adam(lr=0.00008),
                       metrics=['accuracy',metrics.mae, metrics.categorical_accuracy])
# hybrid_model.summary()
plot_model(hybrid_model, to_file='hybrid.png')
plot_model(vgg_model, to_file='vgg.png')


Found 400000 word vectors.


<h1>CNN Image</h1>

In [11]:
IMG_SIZE =224
dataset_dir = 'H:/FYP DATASETS/FYP DATASETS/Crisis/'

def load_img(img):
    path = os.path.join(dataset_dir, img)
    rows=224
    columns=224
    img= cv2.resize(cv2.imread(path,cv2.IMREAD_COLOR),(rows,columns),interpolation=cv2.INTER_CUBIC)
    return img

In [12]:
for index, row in train.iterrows():
    train.at[index,'image_path'] = load_img(row['image_path'])

In [13]:
def encode_label(damage):
    # integer encode
    damage = np.array(damage)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(damage)
    # binary encode
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    return onehot_encoded

In [14]:
y = encode_label(train.iloc[:]['damage'])
print(train.iloc[0])

C:\Users\Saad\AppData\Local\conda\conda\envs\tf_gpu\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


text          Sunny days sweeping clouds away #Irma #PublicP...
text_info                                           informative
image_path    [[[222, 122, 4], [219, 125, 6], [221, 131, 12]...
damage                                              mild_damage
Name: 0, dtype: object


In [15]:
print(train.damage.unique())

['mild_damage' 'little_or_no_damage' 'severe_damage'
 'dont_know_or_cant_judge']


In [16]:
train_images = train['image_path'].tolist()
# no need to convert y to list as it is 1 dim encoding takes care of it
train_images = np.array(train_images)
train_text = np.array(train['text'].tolist())

In [17]:
print(cnn_texts_mat.shape)

(320, 100)


In [18]:
start = time.time()

hybrid_history = hybrid_model.fit(x=[train_images,cnn_texts_mat], y=[y,tags],
                           epochs=40,
                           batch_size=10,
                           validation_split=0.2,
                           shuffle=True,
                           verbose=1)
hybrid_time = time.time() - start


Train on 256 samples, validate on 64 samples
Epoch 1/40
256/256 [==============================] - 35s 135ms/step - loss: 2.0695 - Hybrid_Classifier_loss: 1.3787 - Text_Classifier_loss: 0.6908 - Hybrid_Classifier_acc: 0.3242 - Hybrid_Classifier_mean_absolute_error: 0.3643 - Hybrid_Classifier_categorical_accuracy: 0.3242 - Text_Classifier_acc: 0.5547 - Text_Classifier_mean_absolute_error: 0.4902 - Text_Classifier_categorical_accuracy: 1.0000 - val_loss: 2.0203 - val_Hybrid_Classifier_loss: 1.3842 - val_Text_Classifier_loss: 0.6361 - val_Hybrid_Classifier_acc: 0.3281 - val_Hybrid_Classifier_mean_absolute_error: 0.3666 - val_Hybrid_Classifier_categorical_accuracy: 0.3281 - val_Text_Classifier_acc: 0.6563 - val_Text_Classifier_mean_absolute_error: 0.4643 - val_Text_Classifier_categorical_accuracy: 1.0000
Epoch 2/40
256/256 [==============================] - 7s 29ms/step - loss: 1.8703 - Hybrid_Classifier_loss: 1.2905 - Text_Classifier_loss: 0.5798 - Hybrid_Classifier_acc: 0.3750 - Hybrid_C

Epoch 12/40
256/256 [==============================] - 7s 29ms/step - loss: 1.6447 - Hybrid_Classifier_loss: 1.1446 - Text_Classifier_loss: 0.5001 - Hybrid_Classifier_acc: 0.4180 - Hybrid_Classifier_mean_absolute_error: 0.3302 - Hybrid_Classifier_categorical_accuracy: 0.4180 - Text_Classifier_acc: 0.7852 - Text_Classifier_mean_absolute_error: 0.3229 - Text_Classifier_categorical_accuracy: 1.0000 - val_loss: 1.7634 - val_Hybrid_Classifier_loss: 1.1864 - val_Text_Classifier_loss: 0.5769 - val_Hybrid_Classifier_acc: 0.3594 - val_Hybrid_Classifier_mean_absolute_error: 0.3423 - val_Hybrid_Classifier_categorical_accuracy: 0.3594 - val_Text_Classifier_acc: 0.7187 - val_Text_Classifier_mean_absolute_error: 0.3844 - val_Text_Classifier_categorical_accuracy: 1.0000
Epoch 13/40
256/256 [==============================] - 7s 29ms/step - loss: 1.6212 - Hybrid_Classifier_loss: 1.1500 - Text_Classifier_loss: 0.4712 - Hybrid_Classifier_acc: 0.4063 - Hybrid_Classifier_mean_absolute_error: 0.3298 - Hybri

Epoch 23/40
256/256 [==============================] - 7s 29ms/step - loss: 1.5119 - Hybrid_Classifier_loss: 1.0670 - Text_Classifier_loss: 0.4449 - Hybrid_Classifier_acc: 0.5000 - Hybrid_Classifier_mean_absolute_error: 0.3131 - Hybrid_Classifier_categorical_accuracy: 0.5000 - Text_Classifier_acc: 0.7969 - Text_Classifier_mean_absolute_error: 0.3046 - Text_Classifier_categorical_accuracy: 1.0000 - val_loss: 1.7102 - val_Hybrid_Classifier_loss: 1.1442 - val_Text_Classifier_loss: 0.5660 - val_Hybrid_Classifier_acc: 0.3906 - val_Hybrid_Classifier_mean_absolute_error: 0.3345 - val_Hybrid_Classifier_categorical_accuracy: 0.3906 - val_Text_Classifier_acc: 0.7031 - val_Text_Classifier_mean_absolute_error: 0.3721 - val_Text_Classifier_categorical_accuracy: 1.0000
Epoch 24/40
256/256 [==============================] - 7s 29ms/step - loss: 1.4822 - Hybrid_Classifier_loss: 1.0084 - Text_Classifier_loss: 0.4739 - Hybrid_Classifier_acc: 0.6211 - Hybrid_Classifier_mean_absolute_error: 0.3037 - Hybri

Epoch 34/40
256/256 [==============================] - 7s 29ms/step - loss: 1.3263 - Hybrid_Classifier_loss: 0.9472 - Text_Classifier_loss: 0.3791 - Hybrid_Classifier_acc: 0.6328 - Hybrid_Classifier_mean_absolute_error: 0.2876 - Hybrid_Classifier_categorical_accuracy: 0.6328 - Text_Classifier_acc: 0.8242 - Text_Classifier_mean_absolute_error: 0.2700 - Text_Classifier_categorical_accuracy: 1.0000 - val_loss: 1.6648 - val_Hybrid_Classifier_loss: 1.1152 - val_Text_Classifier_loss: 0.5496 - val_Hybrid_Classifier_acc: 0.4375 - val_Hybrid_Classifier_mean_absolute_error: 0.3257 - val_Hybrid_Classifier_categorical_accuracy: 0.4375 - val_Text_Classifier_acc: 0.7500 - val_Text_Classifier_mean_absolute_error: 0.3546 - val_Text_Classifier_categorical_accuracy: 1.0000
Epoch 35/40
256/256 [==============================] - 7s 29ms/step - loss: 1.3313 - Hybrid_Classifier_loss: 0.9327 - Text_Classifier_loss: 0.3986 - Hybrid_Classifier_acc: 0.6055 - Hybrid_Classifier_mean_absolute_error: 0.2836 - Hybri

In [19]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

In [20]:
start = time.time()

vgg_history = vgg_model.fit(x=[train_images], y=[y],
                           epochs=40,
                           batch_size=10,
                           validation_split=0.2,
                           shuffle=True,
                           verbose=1)
vgg_time = time.time() - start

Train on 256 samples, validate on 64 samples
Epoch 1/40


ResourceExhaustedError: OOM when allocating tensor with shape[25088,4096] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node training_1/Adam/mul_132}} = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_1/Adam/sub_14, training_1/Adam/gradients/fc1_1/MatMul_grad/MatMul_1)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [21]:
print('Hybrid train time ' + str(hybrid_time))
print('vgg train time ' + str(vgg_time))

Hybrid train time 326.57368779182434


NameError: name 'vgg_time' is not defined

In [22]:
hybrid_model.save_weights('hybrid_only.h5')
#vgg_model.save_weights('vgg.h5')